In [6]:
from pathlib import Path
import random
from PIL import Image
from tqdm.auto import tqdm

# ==== Настройки ====
SRC_DIR = Path("../../DeepRockSR-2D") / "shuffled2D" / "shuffled2D_train_HR"  # откуда брать исходные HR
OUT_ROOT = Path("../../DeepRockSR-2D_patches")                  # куда класть патчи

HR_TRAIN = OUT_ROOT / "HR_train"
HR_VALID = OUT_ROOT / "HR_valid"
LR_TRAIN = OUT_ROOT / "LR_train"
LR_VALID = OUT_ROOT / "LR_valid"

PATCH_SIZE = 96           # размер HR-патча
SCALE = 4                 # x4 → внутренний LR будет 96/4 = 24
PATCHES_PER_IMAGE = 50    # всего патчей на изображение
TRAIN_PER_IMAGE = 40      # сколько из них в train
VALID_PER_IMAGE = 10      # сколько в valid (должно равняться PATCHES_PER_IMAGE - TRAIN_PER_IMAGE)

assert TRAIN_PER_IMAGE + VALID_PER_IMAGE == PATCHES_PER_IMAGE

RNG_SEED = 42
random.seed(RNG_SEED)

# ==== Подготовка папок ====
for d in [HR_TRAIN, HR_VALID, LR_TRAIN, LR_VALID]:
    d.mkdir(parents=True, exist_ok=True)

VALID_EXTS = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp"}

image_paths = sorted(
    [p for p in SRC_DIR.iterdir() if p.suffix.lower() in VALID_EXTS]
)

print(f"Найдено HR-изображений: {len(image_paths)}")

global_patch_idx = 0

for img_path in tqdm(image_paths, desc="Обработка изображений"):
    img = Image.open(img_path)
    # Если нужно принудительно ч/б:
    # img = img.convert("L")

    w, h = img.size

    if w < PATCH_SIZE or h < PATCH_SIZE:
        print(f"Пропуск {img_path.name}: размер {w}x{h} меньше {PATCH_SIZE}x{PATCH_SIZE}")
        continue

    base_stem = img_path.stem

    # Чтобы патчи train/valid были случайными, перетасуем индексы 0..49
    local_indices = list(range(PATCHES_PER_IMAGE))
    random.shuffle(local_indices)

    # Первые TRAIN_PER_IMAGE — train, оставшиеся — valid
    train_ids = set(local_indices[:TRAIN_PER_IMAGE])
    valid_ids = set(local_indices[TRAIN_PER_IMAGE:])

    for local_idx in range(PATCHES_PER_IMAGE):
        # Случайная позиция патча
        if w == PATCH_SIZE:
            x = 0
        else:
            x = random.randint(0, w - PATCH_SIZE)

        if h == PATCH_SIZE:
            y = 0
        else:
            y = random.randint(0, h - PATCH_SIZE)

        hr_patch = img.crop((x, y, x + PATCH_SIZE, y + PATCH_SIZE))

        # Внутренний LR 24x24
        lr_inner_size = PATCH_SIZE // SCALE     # 96 → 24
        lr_down = hr_patch.resize((lr_inner_size, lr_inner_size), resample=Image.BICUBIC)
        # Апсемплим назад до 96x96
        lr_patch = lr_down.resize((PATCH_SIZE, PATCH_SIZE), resample=Image.BICUBIC)

        # Имя патча
        patch_name_base = f"{base_stem}_p{global_patch_idx:06d}"
        patch_name_hr = f"{patch_name_base}_HR.png"
        patch_name_lr = f"{patch_name_base}_LR.png"

        if local_idx in train_ids:
            hr_out_path = HR_TRAIN / patch_name_hr
            lr_out_path = LR_TRAIN / patch_name_lr
        else:
            hr_out_path = HR_VALID / patch_name_hr
            lr_out_path = LR_VALID / patch_name_lr

        hr_patch.save(hr_out_path)
        lr_patch.save(lr_out_path)

        global_patch_idx += 1

print(f"Готово. Всего сохранено патчей (train+valid): {global_patch_idx}")
print("HR_train:", HR_TRAIN)
print("HR_valid:", HR_VALID)
print("LR_train:", LR_TRAIN)
print("LR_valid:", LR_VALID)

Найдено HR-изображений: 9600


Обработка изображений:   0%|          | 0/9600 [00:00<?, ?it/s]

Готово. Всего сохранено патчей (train+valid): 480000
HR_train: ..\..\DeepRockSR-2D_patches\HR_train
HR_valid: ..\..\DeepRockSR-2D_patches\HR_valid
LR_train: ..\..\DeepRockSR-2D_patches\LR_train
LR_valid: ..\..\DeepRockSR-2D_patches\LR_valid
